In [10]:
!pip install tika

  Using cached tika-3.1.0-py3-none-any.whl.metadata (15 kB)
Using cached tika-3.1.0-py3-none-any.whl (38 kB)


In [57]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 505184942745.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [15]:
!docker build -t strands-genai .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (2/3)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 629B                                       0.0s
 => [internal] load metadata for public.ecr.aws/lambda/python:3.10         0.1s
 => [internal] load metadata for public.ecr.aws/awsguru/aws-lambda-adapte  0.1s
[+] Building 0.4s (9/15)                                         docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 629B                                       0.0s
 => [internal] load metadata for public.ecr.aws/lambda/python:3.10         0.2s
 => [internal] load metadata for public.ecr.aws/awsguru/aws-lambda-adapte  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B         

In [16]:
!docker tag strands-genai:latest 505184942745.dkr.ecr.us-east-1.amazonaws.com/aws-genai:latest

In [17]:
!docker push 505184942745.dkr.ecr.us-east-1.amazonaws.com/aws-genai:latest

The push refers to repository [505184942745.dkr.ecr.us-east-1.amazonaws.com/aws-genai]

f3451fb4: Preparing 
bf18a086: Preparing 
d3c567f8: Preparing 
c38123c0: Preparing 
7b7edccb: Preparing 
0c3d0ae6: Preparing 
34dcc2e4: Preparing 
5424398f: Preparing 
06912087: Preparing 
4210cee8: Preparing 
c38123c0: Pushed   584.6MB/582MBMBlatest: digest: sha256:79faab17fbb262565a8e21c665a39eafa215a097d7ef193f535a2a8738cb2ee5 size: 2631


In [7]:
import boto3
from tika import parser

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/tika/__init__.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__('pkg_resources').declare_namespace(__name__)


In [8]:
def parse_pdf(document_path:str):
    """
    Parse the pdf to string
    """
    parsed_pdf = parser.from_file(document_path)
    str_content = parsed_pdf['content'] 
    return str_content


In [9]:
parsed_jd= parse_pdf("../data/ML- JobDescription.pdf")
parsed_resume = parse_pdf("../data/resume.pdf")

In [10]:
import json

In [11]:
payload = {
        'job_description': parsed_jd ,
      	'resume': parsed_resume
}
Payload=json.dumps(payload).encode('utf-8')

In [22]:
function_name='llm-strands-backend'
# Invoke Lambda function and stream its response
lambda_client = boto3.client('lambda')

response = lambda_client.invoke_with_response_stream(
    FunctionName=function_name,
    InvocationType='RequestResponse', 
    LogType='None',  
    Payload=Payload
)

In [23]:
response

{'ResponseMetadata': {'RequestId': '3ce38688-dc3f-4c67-b877-e87ac4cd5b5f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 18 Jul 2025 10:36:30 GMT',
   'content-type': 'application/vnd.amazon.eventstream',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3ce38688-dc3f-4c67-b877-e87ac4cd5b5f',
   'x-amz-executed-version': '$LATEST',
   'x-amzn-remapped-content-type': 'application/vnd.awslambda.http-integration-response'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'ExecutedVersion': '$LATEST',
 'ResponseStreamContentType': 'application/vnd.amazon.eventstream',
 'EventStream': <botocore.eventstream.EventStream at 0x7fac876e8640>}

In [24]:
for event in response['EventStream']:
    if 'PayloadChunk' in event:
        event_payload = event['PayloadChunk']['Payload']
        data=event_payload.decode('utf-8','ignore')
        if 'statusCode' not in data:
            #if any(c.isalpha() for c in data):
                 print(data, end="")

        ### Final Recommendation

Based on the comprehensive analysis of Yunlong Jiao's resume, I recommend that we proceed with offering him the Machine Learning Engineer position. Here’s a breakdown of why he is an excellent fit:

1. **Inferred Skills and Technical Proficiency:**
   - Yunlong Jiao has demonstrated strong skills in Python and machine learning libraries such as scikit-learn, numpy, and pandas. His experience with deep learning frameworks like TensorFlow and PyTorch further validates his technical capabilities.
   - His involvement in various high-impact projects, from developing an AI assistant to working on bias mitigation technologies, showcases his versatility and depth in machine learning.

2. **Project Gravity:**
   - The projects he has worked on are significant and real-world, indicating his ability to handle high-stakes responsibilities. His contributions to Amazon and his academic work on deep generative models for speech synthesis highlight his technical prof